[Mask R-CNN for Pytorch](https://pytorch.org/vision/stable/_modules/torchvision/models/detection/mask_rcnn.html)

## Dependencies

In [2]:
import os

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from tqdm import tqdm as loadingbar

In [3]:
from Datasets import MoNuSeg

## Data Preparation

This is going to be difficult... the labels format required by the mask-rcnn will require a lot of attention.

## Model Preparation

In [4]:
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280

model = torchvision.models.detection.MaskRCNN(
    backbone = backbone,
    num_classes = 1
)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\adinb/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:01<00:00, 9.82MB/s]
